In [2]:
import os
import flatten_dict
from datetime import datetime
path = "G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras\\DJI/DJI-Farbkamera_neu/DJI_0001.JPG"

In [10]:
import re
MARKER = {
b"\xff\xe0":"APP0" , # JFIF APP0 segment marker
b"\xff\xe1":"APP1" ,
b"\xff\xe2":"APP2" ,
b"\xff\xe3":"APP3" ,
b"\xff\xe4":"APP4" ,
b"\xff\xe5":"APP5" ,
b"\xff\xe6":"APP6" ,
b"\xff\xe7":"APP7" ,
b"\xff\xe8":"APP8" ,
b"\xff\xe9":"APP9" ,
b"\xff\xea":"APP10",
b"\xff\xeb":"APP11",
b"\xff\xec":"APP12",
b"\xff\xed":"APP13",
b"\xff\xee":"APP14",
b"\xff\xef":"APP15",
b"\xff\xc0":"SOF0" , # Start Of Frame (baseline JPEG) 
b"\xff\xc1":"SOF1" , # Start Of Frame (baseline JPEG) 
b"\xff\xc2":"SOF2" ,
b"\xff\xc3":"SOF3" ,
b"\xff\xc4":"SOF4" ,
b"\xff\xc5":"SOF5" ,
b"\xff\xc6":"SOF6" ,
b"\xff\xc7":"SOF7" ,
b"\xff\xc9":"SOF9" ,
b"\xff\xca":"SOF10",
b"\xff\xcb":"SOF11",
b"\xff\xcd":"SOF13",
b"\xff\xce":"SOF14",
b"\xff\xcf":"SOF15",
b"\xff\xc4":"DHT"  , # Define Huffman Table
b"\xff\xdb":"DQT"  , # Define Quantization Table
b"\xff\xda":"SOS"  ,  # Start of Scan
b"\xff\xc8":"JPG"  ,
b"\xff\xf0":"JPG0" ,
b"\xff\xfd":"JPG13",
b"\xff\xcc":"DAC"  , # Define Arithmetic Table, usually unsupport 
b"\xff\xdc":"DNL"  ,
b"\xff\xdd":"DRI"  , # Define Restart Interval
b"\xff\xde":"DHP"  ,
b"\xff\xdf":"EXP"  ,
b"\xff\xd0":"RST0" ,  # RSTn are used for resync, may be ignored
b"\xff\xd1":"RST1" ,
b"\xff\xd2":"RST2" ,
b"\xff\xd3":"RST3" ,
b"\xff\xd4":"RST4" ,
b"\xff\xd5":"RST5" ,
b"\xff\xd6":"RST6" ,
b"\xff\xd7":"RST7" ,
    
b"\xff\x01":"TEM"  ,
b"\xff\xfe":"COM"  } # Comment
# ffd9 is end of image
           
    

TYPES = {1: {"id":1, "name":"Byte","format":"B","size":1},
        2: {"id":2,  "name":"Ascii","format":"s","size":1},
        3: {"id":3,  "name":"Short","format":"H","size":2},
        4: {"id":4,  "name":"Long","format":"L","size":4},
        5: {"id":5,  "name":"Rational","format":"LL","size":8},
        6: {"id":6,  "name":"SByte","format":"b","size":1},
        7: {"id":7,  "name":"Undefined","format":"B","size":1}, # B
        8: {"id":8,  "name":"SShort","format":"h","size":2},
        9: {"id":9,  "name":"SLong","format":"l","size":4},
        10: {"id":10,"name":"SRational","format":"ll","size":8},
        11: {"id":11,"name":"Float","format":"f","size":4},
        12: {"id":12,"name":"Double","format":"d","size":8},
        }


ENDIAN = {b"II*\x00":{"name":"Little Endian","format":"<","data":b"II*\x00","shortname":"little"},
          b"MM*\x00":{"name":"Big Endian","format":">","data":b"MM*\x00","shortname":"big"}}

IFD_TAG_SIZE = 12 # HHII (TagID (short), Type (short), Count (int),ValueOffset (int) )
NEXT_IFD_OFFSET_SIZE = 4
NUM_OF_INTEROP_SIZE = 2

MARKER_2BYTE = {
    b"\xff\xd8":"SOI"  ,  # Start of Image
    b"\xff\xd9":"EOI"  ,  # End of Image
    b"\xff\xd0":"RST0" ,  # RSTn are used for resync, may be ignored
    b"\xff\xd1":"RST1" ,
    b"\xff\xd2":"RST2" ,
    b"\xff\xd3":"RST3" ,
    b"\xff\xd4":"RST4" ,
    b"\xff\xd5":"RST5" ,
    b"\xff\xd6":"RST6" ,
    b"\xff\xd7":"RST7" ,
    b"\xff\x01":"TEM"  ,
    }

MARKER_2BYTE_REV = {y:x for x,y in MARKER_2BYTE.items()}

MARKER_REV = {y:x for x,y in MARKER.items()}


def find_segments(rawdata):
    """
    A Jpeg file is normally structured as JFIF.
    JFIF has Segments (see MARKER)

    returns the complete File as a Segments-List
    """
    pattern = bytearray(b"..|".join(MARKER.keys()))
    pattern.extend(b"..|")
    pattern.extend(b"|".join(MARKER_2BYTE.keys()))
    cpattern = re.compile(bytes(pattern))
    segments = []
    id = 0
    parent = 0
    for m in cpattern.finditer(rawdata):
        if len(m.group())>=4: 
            seg_length = 256 * m.group()[2] + m.group()[3]
            segment = MARKER[m.group()[:-2]]
        else:
            seg_length = 0
            try:
                segment = MARKER[m.group()]
            except KeyError:
                segment = MARKER_2BYTE[m.group()]
            except: 
                logging.error("find JFIF Segment failed",exc_info=True)
        if segment in ["APP0","APP1"]:
            seg_type = rawdata[m.start()+4:m.start()+8]
        else:
            seg_type = ''

        parent,level = next(((item["id"],item["level"]+1) 
                            for item in reversed(segments) 
                                if item["end"] > m.start()), (0,0))
        segments.append({"id":id,
            "segment": segment,
            "pos":m.start(),
            "end" : m.start()+seg_length+2,
            "type": seg_type,             
            "parent": parent, 
            "level": level, 
            "data": rawdata[m.start() : m.start() + seg_length+2]   })
        id += 1
    return segments

filepath = "C:/Users/marti/Downloads/dji_thermal_sdk_v1.2_20211209/dataset/XTS/DJI_0001_R.jpg"
with open(filepath,"rb") as f:
    rawdata = f.read()
seg = find_segments(rawdata) 
for s in seg:
    print(s["id"],s["segment"],s["type"],s["pos"],s["end"],s["parent"],s["level"])
 

0 SOI  0 2 0 0
1 APP1 b'Exif' 2 17740 0 0
2 SOI  440 442 1 1
3 APP0 b'JFIF' 442 460 1 1
4 DQT  460 594 1 1
5 DHT  594 1014 1 1
6 SOF0  1014 1033 1 1
7 DRI  1033 1039 1 1
8 SOS  1039 1053 1 1
9 EOI  17738 17740 1 1
10 APP2  17740 19004 0 0
11 APP3  19004 51776 0 0
12 COM  20556 86092 11 1
13 COM  20560 20817 12 2
14 TEM  20565 85848 13 3
15 TEM  20569 85853 14 4
16 SOF0  21681 87155 15 5
17 SOF0  21685 87160 16 6
18 SOF1  21689 87164 17 7
19 SOF2  21693 87169 18 8
20 SOF2  21697 87174 19 9
21 SOF3  21701 87178 20 10
22 DHT  21705 87183 21 11
23 DHT  21709 87188 22 12
24 SOF5  21713 87192 23 13
25 SOF6  21717 87197 24 14
26 SOF6  21721 87202 25 15
27 SOF7  21725 87206 26 16
28 JPG  21729 87211 27 17
29 JPG  21733 87216 28 18
30 SOF9  21737 87220 29 19
31 SOF10  21741 87225 30 20
32 SOF10  21745 87230 31 21
33 SOF11  21749 87234 32 22
34 DAC  21753 87239 33 23
35 DAC  21757 87243 34 24
36 SOF13  21761 87248 35 25
37 SOF13  21765 87253 36 26
38 SOF14  21769 87257 37 27
39 SOF15  21773 8726

In [ ]:
pointer = 0
offset = 2 # this is the length of MARKER. 
if rawdata[pointer:pointer+2]==b'\xff\xd8':
    print("THIS IS A JFIF FILE") 
    pointer = 2
filelength = len(rawdata)
while pointer < filelength:
    if rawdata[pointer:pointer+2] in MARKER.keys():
        print("position:",pointer)
        print("MARKER found!", MARKER[rawdata[pointer:pointer+2]])
        print (rawdata[pointer:pointer+2])
        pointer = offset + pointer + 256 * rawdata[pointer+2] + rawdata[pointer+3]
        #print (rawdata[pointer+offset:pointer+offset+2])
    elif rawdata[pointer:pointer+2] in MARKER_2BYTE.keys():
        print("2 BYTE MARKER found!", MARKER_2BYTE[rawdata[pointer:pointer+2]])
        pointer = offset + pointer
        print("position:",pointer)
    else:   
        print("NOTHING FOUND!")
        print (rawdata[pointer-10:pointer+offset+10])
        
    
    

In [4]:
t = os.path.getmtime(path)
datetime.fromtimestamp(t).strftime("%Y-%m-%d %H:%M:%S ")

'2020-05-14 13:41:46 '

In [5]:
rational = (72, 1,2)

In [6]:
def rational2float(rational):
    if len(rational) == 2:
        return(rational[0]/rational[1])
    else:
        return(rational)

In [7]:
rational2float(rational)

(72, 1, 2)

In [8]:
gpsdatetime = b'1970:01:01 00:03:1'
subsec = b'88300'

In [9]:
datetime.strptime((gpsdatetime +b"."+ subsec).decode(),"%Y:%m:%d %H:%M:%S.%f")

datetime.datetime(1970, 1, 1, 0, 3, 1, 883000)

In [10]:
if b'-21345' in [b"S",b"South"] or b"-2345" < b"0":
    print("True")

True


In [11]:
b'234'.decode()

'234'

In [12]:
import xmltodict

xmp_str = b'<rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">\r\n\
<rdf:Description rdf:about=""\r\n\
xmlns:Camera="http://pix4d.com/camera/1.0/"\r\n\
xmlns:FLIR="http://ns.flir.com/xmp/1.0/"\r\n\
<Camera:BandName>\r\n<rdf:Seq>\r\n\
<rdf:li>LWIR</rdf:li>\r\n\
</rdf:Seq>\r\n</Camera:BandName>\r\n\
<Camera:CentralWavelength>\r\n\
<rdf:Seq>\r\n<rdf:li>10000</rdf:li>\r\n</rdf:Seq>\r\n\
</Camera:CentralWavelength>\r\n\
<Camera:WavelengthFWHM>\r\n<rdf:Seq>\r\n<rdf:li>4500</rdf:li>\r\n</rdf:Seq>\r\n</Camera:WavelengthFWHM>\r\n\
<Camera:TlinearGain>0.00</Camera:TlinearGain>\r\n\
<Camera:Yaw>0/100</Camera:Yaw>\r\n\
<Camera:Pitch>0/100</Camera:Pitch>\r\n\
<Camera:Roll>0/100</Camera:Roll>\r\n\
<Camera:GPSXYAccuracy>0.00</Camera:GPSXYAccuracy>\r\n\
<Camera:GPSZAccuracy>0.00</Camera:GPSZAccuracy>\r\n\
<Camera:GyroRate>0.00</Camera:GyroRate>\r\n\
<Camera:DetectorBitDepth>16</Camera:DetectorBitDepth>\r\n\
<Camera:IsNormalized>0</Camera:IsNormalized>\r\n\
<FLIR:MAVVersionID>0.3.0.0</FLIR:MAVVersionID>\r\n\
<FLIR:MAVComponentID>100</FLIR:MAVComponentID>\r\n\
<FLIR:MAVRelativeAltitude>-327/1000</FLIR:MAVRelativeAltitude>\r\n\
<FLIR:MAVRateOfClimbRef>M</FLIR:MAVRateOfClimbRef>\r\n\
<FLIR:MAVRateOfClimb>0/1000</FLIR:MAVRateOfClimb>\r\n\
<FLIR:MAVYaw>2009/100</FLIR:MAVYaw>\r\n\
<FLIR:MAVPitch>-136/100</FLIR:MAVPitch>\r\n\
<FLIR:MAVRoll>-102/100</FLIR:MAVRoll>\r\n\
<FLIR:MAVYawRate>4/100</FLIR:MAVYawRate>\r\n\
<FLIR:MAVPitchRate>-10/100</FLIR:MAVPitchRate>\r\n\
<FLIR:MAVRollRate>-10/100</FLIR:MAVRollRate>\r\n\
</rdf:Description>\r\n\
</rdf:RDF>'

xmp_od = xmltodict.parse(xmp_str)
xmp = xmp_od.get("rdf:RDF").get("rdf:Description")

ExpatError: not well-formed (invalid token): line 5, column 0

In [14]:
xmp_str2 = b'<rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"\r\nxmlns:Camera="http://pix4d.com/camera/1.0/"\r\nxmlns:FLIR="http://ns.flir.com/xmp/1.0/">\r\n<rdf:Description rdf:about="">\r\n<Camera:BandName>\r\n<rdf:Seq>\r\n<rdf:li>LWIR</rdf:li>\r\n</rdf:Seq>\r\n</Camera:BandName>\r\n<Camera:CentralWavelength>\r\n<rdf:Seq>\r\n<rdf:li>10000</rdf:li>\r\n</rdf:Seq>\r\n</Camera:CentralWavelength>\r\n<Camera:WavelengthFWHM>\r\n<rdf:Seq>\r\n<rdf:li>4500</rdf:li>\r\n</rdf:Seq>\r\n</Camera:WavelengthFWHM>\r\n<Camera:TlinearGain>0.00</Camera:TlinearGain>\r\n<Camera:IsNormalized>0</Camera:IsNormalized>\r\n<Camera:Yaw>75/100</Camera:Yaw>\r\n<Camera:Pitch>-4500/100</Camera:Pitch>\r\n<Camera:Roll>0/100</Camera:Roll>\r\n<Camera:GPSXYAccuracy>0.00</Camera:GPSXYAccuracy>\r\n<Camera:GPSZAccuracy>0.00</Camera:GPSZAccuracy>\r\n<Camera:GyroRate>0.00</Camera:GyroRate>\r\n<Camera:DetectorBitDepth>16</Camera:DetectorBitDepth>\r\n<FLIR:MAVVersionID>0.3.0.0</FLIR:MAVVersionID>\r\n<FLIR:MAVComponentID>100</FLIR:MAVComponentID>\r\n<FLIR:MAVRelativeAltitude>432/1000</FLIR:MAVRelativeAltitude>\r\n<FLIR:MAVRateOfClimbRef>M</FLIR:MAVRateOfClimbRef>\r\n<FLIR:MAVRateOfClimb>-180/1000</FLIR:MAVRateOfClimb>\r\n<FLIR:MAVYaw>2819/100</FLIR:MAVYaw>\r\n<FLIR:MAVPitch>-7/100</FLIR:MAVPitch>\r\n<FLIR:MAVRoll>-52/100</FLIR:MAVRoll>\r\n<FLIR:MAVYawRate>10/100</FLIR:MAVYawRate>\r\n<FLIR:MAVPitchRate>-50/100</FLIR:MAVPitchRate>\r\n<FLIR:MAVRollRate>-9/100</FLIR:MAVRollRate>\r\n</rdf:Description>\r\n</rdf:RDF>'

xmp_od2 = xmltodict.parse(xmp_str2)
xmp_2 = xmp_od2.get("rdf:RDF").get("rdf:Description")
xmp_od2

OrderedDict([('rdf:RDF',
              OrderedDict([('@xmlns:rdf',
                            'http://www.w3.org/1999/02/22-rdf-syntax-ns#'),
                           ('@xmlns:Camera', 'http://pix4d.com/camera/1.0/'),
                           ('@xmlns:FLIR', 'http://ns.flir.com/xmp/1.0/'),
                           ('rdf:Description',
                            OrderedDict([('@rdf:about', ''),
                                         ('Camera:BandName',
                                          OrderedDict([('rdf:Seq',
                                                        OrderedDict([('rdf:li',
                                                                      'LWIR')]))])),
                                         ('Camera:CentralWavelength',
                                          OrderedDict([('rdf:Seq',
                                                        OrderedDict([('rdf:li',
                                                                      '10000')]))])),


In [15]:
import nested

def makecaseinsensitive(k,v,**kwargs): 
    '''
    this callback function makes all keys of the dicts insensitive 
    '''
    if type(v) == dict]:
        k = k.lower()
    elif type(v) == list and len(v) == 1 and type(v[0]) == dict:
        if list(v[0].keys()) == [str(i) for i in range(0,len(v[0]))]:
            v = list(v[0].values())
        else:
            v = v[0]
    return v

n = nested.Nested(xmp_od2,)
n.data

SyntaxError: unmatched ']' (<ipython-input-15-bcd45ff9d5ca>, line 7)

In [16]:


from collections import Mapping

class CaseInsensitiveDict(Mapping):
    def __init__(self, d):
        self._d = d
        self._s = dict((k.lower(), k) for k in d)
    def __contains__(self, k):
        return k.lower() in self._s
    def __len__(self):
        return len(self._s)
    def __iter__(self):
        return iter(self._s)
    def __getitem__(self, k):
        return self._d[self._s[k.lower()]]
    def actual_key_case(self, k):
        return self._s.get(k.lower())

<ipython-input-16-c379620bb07e>:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping


In [17]:
from functools import reduce  # forward compatibility for Python 3
import operator

def dict_generator(indict, pre=None):
    pre = pre[:] if pre else []
    if isinstance(indict, dict):
        for key, value in indict.items():
            if isinstance(value, dict):
                for d in dict_generator(value, pre + [key]):
                    yield d
            elif isinstance(value, list) or isinstance(value, tuple):
                for v in value:
                    for d in dict_generator(v, pre + [key]):
                        yield d
            else:
                yield pre + [key, value]
    else:
        yield pre + [indict]
        
def getFromDict(dataDict, mapList):
    return reduce(operator.getitem, mapList, dataDict)

def setInDict(dataDict, mapList, value):
    getFromDict(dataDict, mapList[:-1])[mapList[-1]] = value

def buildDict():
    
dictneu={}
pre = []
for i in dict_generator(xmp_od2):    
    try:
        setInDict(dictneu,pre,i[-1])
    
    
    for j in i[:-2]:
        pre.append(j)
        print(j,pre)
        setInDict(dictneu,pre,{})
    pre.append(i[-1])
    setInDict(dictneu,pre,i[-1])
    del pre[-1]
    #pre = []
    #print("last",i[-1])
dictneu

IndentationError: expected an indented block (<ipython-input-17-5058acb006f9>, line 28)

In [18]:
my_dict = {'KEY1': {"InneR":"Hello","In2":{"BLA":"Foo"}}, 'Key2': "World"} 
def lowerkeys(dic):
    return dict((k.lower(), v) for k, v in dic.items()) 
lowerkeys(my_dict)

{'key1': {'InneR': 'Hello', 'In2': {'BLA': 'Foo'}}, 'key2': 'World'}

2.1

In [23]:
flatten_dict.flatten(my_dict)

{('KEY1', 'InneR'): 'Hello', ('KEY1', 'In2', 'BLA'): 'Foo', ('Key2',): 'World'}

In [19]:
def deep_get(dictionary, *keys):
    return reduce(lambda d, key: d.get(key) if d else None, keys, dictionary)
deep_get(my_dict, "KEY1","In22")

NameError: name 'reduce' is not defined

In [20]:
import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)
flatten(my_dict)

<ipython-input-20-021befedaaaf>:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  if isinstance(v, collections.MutableMapping):


{'KEY1_InneR': 'Hello', 'KEY1_In2_BLA': 'Foo', 'Key2': 'World'}

In [21]:
dictneu = {}
for i in dict_generator(xmp_od2): 
    print("keys:", i[:-1], "value:", i[-1])
    for j in i[:-1]:
    if not j in dictneu:

IndentationError: expected an indented block (<ipython-input-21-664b4af0f75a>, line 5)

In [ ]:
xmp_od2

In [ ]:
mydic = {"a":{"b":{"c":"Wert"}}}
mylist = ["a","b","c"]
pre = []
for i in mylist:
    pre.append(i)
    print (pre)

In [ ]:
if "a" in mydic:
    print("true")

In [ ]:
rdf = xmp_od2.get(next(iter(xmp_od2)))
def get_ci(dic,key_ci):
    for k,v in dic.items():
        if k.lower() == key_ci:
            return v
        


In [ ]:
get_ci(get_ci(xmp_od2,"rdf:rdf"),"rdf:description")

In [ ]:
newdict = {}
for i in dict_generator(xmp_od2):
    for j in i:
        newdict[j]= 
#import string

#dict(zip(map(str.lower,xmp_od2.keys()),xmp_od2.values()))


In [ ]:
if xmp_str.find(b'http://ns.flir.com/xmp/1.0/"\r'):
    if xmp_str.find(b'http://ns.flir.com/xmp/1.0/">\r') == -1:
        xmp_str = xmp_str.replace(b'http://ns.flir.com/xmp/1.0/"\r',b'http://ns.flir.com/xmp/1.0/">\r')
        

In [ ]:
xmp_str

In [ ]:
class CaseInsensitiveDict(dict):
    def __setitem__(self, key, value):
        super(CaseInsensitiveDict, self).__setitem__(key.lower(), value)

    def __getitem__(self, key):
        return super(CaseInsensitiveDict, self).__getitem__(key.lower())

In [ ]:
class CaseInsensitiveDict(dict):
    @classmethod
    def _k(cls, key):
        return key.lower() if isinstance(key, basestring) else key

    def __init__(self, *args, **kwargs):
        super(CaseInsensitiveDict, self).__init__(*args, **kwargs)
        self._convert_keys()
    def __getitem__(self, key):
        return super(CaseInsensitiveDict, self).__getitem__(self.__class__._k(key))
    def __setitem__(self, key, value):
        super(CaseInsensitiveDict, self).__setitem__(self.__class__._k(key), value)
    def __delitem__(self, key):
        return super(CaseInsensitiveDict, self).__delitem__(self.__class__._k(key))
    def __contains__(self, key):
        return super(CaseInsensitiveDict, self).__contains__(self.__class__._k(key))
    def has_key(self, key):
        return super(CaseInsensitiveDict, self).has_key(self.__class__._k(key))
    def pop(self, key, *args, **kwargs):
        return super(CaseInsensitiveDict, self).pop(self.__class__._k(key), *args, **kwargs)
    def get(self, key, *args, **kwargs):
        return super(CaseInsensitiveDict, self).get(self.__class__._k(key), *args, **kwargs)
    def setdefault(self, key, *args, **kwargs):
        return super(CaseInsensitiveDict, self).setdefault(self.__class__._k(key), *args, **kwargs)
    def update(self, E={}, **F):
        super(CaseInsensitiveDict, self).update(self.__class__(E))
        super(CaseInsensitiveDict, self).update(self.__class__(**F))
    def _convert_keys(self):
        for k in list(self.keys()):
            v = super(CaseInsensitiveDict, self).pop(k)
            self.__setitem__(k, v)

In [ ]:

CaseInsensitiveDict(xmp_2)
#for i in xmp_2:
#    print(i)

In [ ]:
from bs4 import BeautifulSoup
xmp_bs = BeautifulSoup(xmp_str,"lxml")
xmp_od = xmltodict.parse(bytes(str(xmp_bs.find("rdf:rdf")),"utf-8"))
xmpbs = xmp_od.get("rdf:rdf").get("rdf:description")
xmpbs

In [ ]:
xmpbs.get("flir:mavroll")

In [ ]:
bytes(str(xmp_bs.find("rdf:rdf")),"utf-8")

In [ ]:
xmp_od = xmltodict.parse(xmp)
xmp = xmp_od.get("rdf:RDF").get("rdf:Description")
